In [ ]:
import os,sys
sys.path.append('/home/zkd/文档/ION/Desktop/241209_393锁定/code')
from Photon_infere import *

In [ ]:
os.chdir(r'/mnt/webdav1/2.3_DI-QKD/0.0_Charlie/raw_data/0226_alice_EIT_radial&axial')

In [ ]:
fold=os.listdir()
for i,_ in enumerate(fold):
    print(i,_)

## Gamma Distribution

In [ ]:
sys_json=read_sys_json('sys.json')

# 荧光计数窗口
window_start=sys_json['window_start(ps)']
window_end=sys_json['window_end(ps)']
# 分析数据截至时间
period=sys_json['period(s)']
time_start=sys_json['time_start(s)']
time_stop=sys_json['time_stop(s)']
#脉冲
pulse_num=sys_json['pulse_num']
pulse_interval=sys_json['pulse_interval(ps)'] 

# turn=sys_json['turns'][3]

# 移动窗口
window_start_list=np.array(sys_json['window_start_list(ns)'])*1e3 #ps
window_len=sys_json['window_len(ns)']*1e3 #ps
folds=sys_json['turns']
out_csv='gamma_res_time_t_{:.0f}ns.csv'.format(window_len/1e3)
print('outfile name: ',out_csv)

In [ ]:

for turn in folds:
    # 分析数据截至时间
    for window_start in window_start_list:
        window_end=window_start+window_len

        pmt1_df=read_id900_binary(turn,3)
        pmt2_df=read_id900_binary(turn,4)
        ## 时间限制
        pmt1_df=pmt1_df[(pmt1_df['time_satmp']>=time_start/period)&(pmt1_df['time_satmp']<=time_stop/period)].copy()
        pmt2_df=pmt2_df[(pmt2_df['time_satmp']>=time_start/period)&(pmt2_df['time_satmp']<=time_stop/period)].copy()
        ## 计算时间窗内光子数
        pmt1_index=window_index(pmt1_df,window_start,window_end,pulse_num=pulse_num,pulse_interval=pulse_interval)
        pmt2_index=window_index(pmt2_df,window_start,window_end,pulse_num=pulse_num,pulse_interval=pulse_interval)
        pmt1_count=pmt1_df[pmt1_index]['start_index'].value_counts().sort_index(ascending=True)
        pmt2_count=pmt2_df[pmt2_index]['start_index'].value_counts().sort_index(ascending=True)
        PMT1_SUM=pmt1_count.sum()
        PMT2_SUM=pmt2_count.sum()
        print('pmt1:{}, pmt2:{}'.format(PMT1_SUM,PMT2_SUM))
        ## 写入文件
        try:
            with open(out_csv, 'r') as f:
                pass
        except FileNotFoundError:
            # 文件不存在，写入标题
            with open(out_csv, 'w') as f:
                f.write('turn,state,start(ps),end(ps),pmt1,pmt2\n')

        with open(out_csv,'a') as f:
            f.write('{a},origin,{t0:.0f},{t1:.0f},{pmt1:.0f},{pmt2:.0f}\n'.format(a=turn.split('_')[0],t0=window_start,t1=window_end,pmt1=PMT1_SUM,pmt2=PMT2_SUM))



## Get Gamma & window

In [ ]:
# out_csv='gamma_res_time_t.csv'
res_data=pd.read_csv(out_csv)
count2gamma(res_data)
res_data.sort_values(by='turn',inplace=True)
origin_df=res_data[res_data['state']=='origin']
choice_df=res_data[res_data['state']=='select']
origin_df['phase'].unique()

In [ ]:
origin_df['delta']=origin_df['end(ps)']-origin_df['start(ps)']
out_dict={'t0':[],'gamma':[],'gamma_err':[],'phase_offset':[],'phase_offset_err':[],'pmt1':[],'pmt2':[]}
for start_t in origin_df['start(ps)'].unique():
    df00=origin_df[origin_df['start(ps)']==start_t]
    delta_t=df00['start(ps)'].values[0]/1000
    print('window len :{:.0f}ns'.format(delta_t),end=' | ')
    param,param_err=estimate_gamma(df00,model_func=model_func_count_offset)
    # plt.savefig('gamma_fit_{}.png'.format(delta_t))
    plt.close()
    out_dict['t0'].append(df00['start(ps)'].values[0])
    out_dict['gamma'].append(param[0])
    out_dict['gamma_err'].append(param_err[0])
    out_dict['phase_offset'].append(param[1])
    out_dict['phase_offset_err'].append(param_err[1])
    out_dict['pmt1'].append(df00['pmt1'].sum())
    out_dict['pmt2'].append(df00['pmt2'].sum())

out_dict=pd.DataFrame(data=out_dict)
out_dict['gamma']=np.abs(out_dict['gamma'])
out_dict.sort_values(by='t0',inplace=True)
title_txt='{:.0f}ns window move, MAX gamma={:.2%}'.format(window_len*1e-3,np.max(out_dict['gamma']))
plt.subplot(211)
plt.errorbar(out_dict['t0']/1000,out_dict['gamma']*100,yerr=out_dict['gamma_err']*100,fmt='-.o',label='experimental data',capsize=5)
plt.xlabel('start time [ns]')
plt.ylabel('gamma [%]')
plt.title(title_txt)
plt.grid()
plt.subplot(212)

start_list=(out_dict['t0'].values-out_dict['t0'].values[0])/1000
phase=out_dict['phase_offset'].values
phase=np.unwrap(phase)
liner=lambda x, a, b: 2*np.pi*a*x+b
if np.isinf(phase).any():
    param,covariance=curve_fit(liner, start_list, phase)
else:
    param,covariance=curve_fit(liner, start_list, phase,sigma=out_dict['phase_offset_err'])
est_phase=liner(start_list, *param)
r_squared=1-np.sum((est_phase-phase)**2)/np.sum((phase-np.mean(phase))**2)
plt.errorbar(start_list,phase*180/np.pi,yerr=out_dict['phase_offset_err']*180/np.pi,fmt='o',label='experimental data',capsize=5)
plt.plot(start_list,est_phase*180/np.pi,'-.',label='fit {:.1%}±{:.1%}'.format(np.abs(param[0]),np.sqrt(np.diag(covariance))[0]))
plt.xlabel('start time [ns]')
plt.ylabel('phase offset [deg]')
plt.title('LS estimae $f$: {:.3f}±{:.3f} MHz, $R^2=${:.1%}'.format(np.abs(param[0])*1e3,np.sqrt(np.diag(covariance))[0]*1e3,r_squared))
plt.grid()
plt.tight_layout()
plt.savefig('df_{:.0f}ns_window_fit.jpeg'.format(window_len*1e-3),dpi=300)
plt.show()

In [ ]:
plt.plot(out_dict['t0']*1e-3,out_dict['pmt1'],'-o',label='pmt1')
plt.plot(out_dict['t0']*1e-3,out_dict['pmt2'],'-o',label='pmt2')
plt.plot(out_dict['t0']*1e-3,(out_dict['pmt1']+out_dict['pmt2'])/2,'-.x',label='mean')
plt.title('count with {:.0f}ns window move'.format(window_len*1e-3))
plt.xlabel('start time[ns]')
plt.ylabel('count')
plt.legend()
plt.grid()
plt.show()